**Aspect Extraction Library Import**

In [22]:
!pip install pyabsa


In [23]:

import pandas as pd
from pyabsa import ATEPCCheckpointManager

In [24]:
# Initialize the aspect extractor
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(
    checkpoint='english',
    auto_device=True
)

def extract_aspects(text):
    # Perform aspect extraction
    result = aspect_extractor.extract_aspect(
        inference_source=[text],
        pred_sentiment=True
    )
    # Extract and return the aspects
    return result[0]['aspect']

# Single string input
text = "Dictatorship naturally arises out of democracy"

# Extract aspects from the single string
aspects = extract_aspects(text)

# Print the extracted aspects
print("Extracted Aspects:", aspects)

[2024-08-04 11:42:22] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-04 11:42:23] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-04 11:42:23] (2.4.1.post1) Downloading checkpoint:english 
[2024-08-04 11:42:23] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2024-08-04 11:42:23] (2.4.1.post1) Checkpoint already downloaded, skip
[2024-08-04 11:42:23] (2.4.1.post1) Load aspect extractor from checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43
[2024-08-04 11:42:23] (2.4.1.post1) config: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.config
[2024-08-04 11:42:23] (2.4.1.post1) state_dict: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apc

In [25]:
print(type(aspects))

<class 'list'>


**Installing Transformers for Model to Data Product Implementation**

In [26]:
!pip install transformers[sentencepiece]


**Importing Pipelines**

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline

# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

In [28]:
sentence = "Dictatorship naturally arises out of democracy"
print(f"Sentence: {sentence}")
print()


aspect = " ".join(aspects)

# Tokenize and get model predictions for the aspect-based sentiment analysis
inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
outputs = absa_model(**inputs)
probs = F.softmax(outputs.logits, dim=1)
probs = probs.detach().numpy()[0]

# Define labels
labels = ["Sarcastic", "Non Sarcastic"]

# Print the probabilities for each label
print(f"Sentiment of aspect '{aspect}' is:")
for prob, label in zip(probs, labels):
  print(f"Label {label}: {prob}")

# Determine and print the label with the highest probability
max_prob_index = probs.argmax()
max_prob_label = labels[max_prob_index]
print(f"\nFinal Result: The aspect '{aspect}' is {max_prob_label} with a probability of {probs[max_prob_index]:.4f}")

Sentence: Dictatorship naturally arises out of democracy

Sentiment of aspect 'democracy' is:
Label Sarcastic: 0.6769244074821472
Label Non Sarcastic: 0.3117344379425049

Final Result: The aspect 'democracy' is Sarcastic with a probability of 0.6769
